2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [2]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
torch.cuda.is_available()
torch.cuda.set_device(0)
import atd2022

In [3]:
mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.train.batch_size = 60
mnist_args.predict.batch_size = 100

mnist_args.train_epochs = 10
mnist_args.lr = 0.0001
mnist_args.criterion = 'CE'
mnist_args.use_gpu = True

input_channel = 1
input_height = 28
input_width = 28
kernel_size = 9
knpp = 10
knpp2 = 20

mnist_args.layers=[
    ('specific', (input_channel, input_height*input_width, kernel_size, knpp)),
    ('specific', (knpp, input_height*input_width, kernel_size, knpp2)),
    ('linear', (knpp2*input_height*input_width, 10, (1,2,3))),
    ('softmax', (1))
]

In [15]:
atd_args = dotdict()
atd_args.train = dotdict()

atd_args.name = 'atd'
atd_args.train.batch_size = 30
atd_args.predict_len = 1
atd_args.history_len = 5

atd_args.train_epochs= 100
atd_args.lr = 0.001
atd_args.criterion = 'L1'
atd_args.use_gpu = True

input_channel = 1
input_height = atd_args.history_len
input_width = 5200
kernel_size = 6
knpp = 6
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('agnostic', (input_channel, input_height*input_width, kernel_size, knpp)),
    ('agnostic', (knpp, input_height*input_width, kernel_size, knpp2)),
    ('linear', (knpp2*input_height, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [16]:
wt_args = dotdict()
wt_args.train = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train.batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.use_gpu = True

input_channel = 1
input_height = wt_args.history_len
input_width = 1400
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', (input_channel, input_height*input_width, kernel_size, knpp)),
    ('specific', (knpp, input_height*input_width, kernel_size, knpp2)),
    ('linear', (knpp2*input_height, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [17]:
lat_args = dotdict()
lat_args.train = dotdict()

lat_args.name = 'lat'
lat_args.train.batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.use_gpu = True

input_channel = 1
input_height = lat_args.history_len
input_width = 207
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', (input_channel, input_height*input_width, kernel_size, knpp)),
    ('specific', (knpp, input_height*input_width, kernel_size, knpp2)),
    ('linear', (knpp2*input_height, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [19]:
args = lat_args
# data = atd2022.io.read_csv()
# data = None
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
data = data.head(200)

driver = ABC_Driver(args, data)
driver.train()

Use GPU: cuda:None
epoch: 0, train_loss: 0.6472602912357875
epoch: 1, train_loss: 0.605058661528996
epoch: 2, train_loss: 0.5633579322269985
epoch: 3, train_loss: 0.5191823840141296
epoch: 4, train_loss: 0.4716698569910867
epoch: 5, train_loss: 0.4232517736298697
epoch: 6, train_loss: 0.37752742426736013
epoch: 7, train_loss: 0.33737662008830477
epoch: 8, train_loss: 0.30188181144850595
epoch: 9, train_loss: 0.2710962189095361


In [20]:
# driver.metric()

In [12]:
train_pred = driver.predict(driver.data_loader.train).to(float).round()
train_pred

tensor([[[[  2.,   1.,  20.,  ...,  28.,  56.,   1.]]],


        [[[  1.,   0.,  -1.,  ...,  54.,  97.,   0.]]],


        [[[  3.,   1.,  21.,  ...,  36.,  76.,   0.]]],


        ...,


        [[[ 16.,   5.,   4.,  ..., 119., 152.,   1.]]],


        [[[ 10.,   5.,   2.,  ...,  80., 178.,   4.]]],


        [[[ 23.,   6.,   8.,  ..., 115., 134.,   2.]]]], dtype=torch.float64)

In [13]:
pred = driver.predict().to(float)
pred

tensor([[[[  5.7886,   2.5833,   2.7232,  ...,  51.8400, 157.8637,   3.4961]]]],
       dtype=torch.float64)

In [14]:
data.mean()

Region  Event
AA      01        10.700
        02         4.230
        03         7.100
        04        24.525
        05         5.190
                  ...   
ZI      16        45.055
        17       261.165
        18        94.235
        19       199.690
        20         1.025
Length: 5200, dtype: float64